## Demographic

In [249]:
from os import sep
import pandas as pd
import numpy as np
from classes import User, Movie
from scipy.stats.stats import pearsonr
import tqdm

# Genres
generos = pd.read_csv("data/genre.txt", names=["genre_id", "genre_name"], sep="\t")


# Read users
users_df = pd.read_csv(
    "data/users.txt", names=["user_id", "age", "gender", "occupation"], sep="\t"
)

# Read movies
all_genre = generos.genre_name.values.tolist()
all_genre = ["movie_id"] + all_genre + ["title"]
films = []
films_df = pd.read_csv("data/items.txt", names=all_genre, sep="\t")
for idx, row in films_df.iterrows():
    films.append(Movie(row["movie_id"], row[all_genre].values.tolist(), row["title"]))


# Ratings
ratings = pd.read_csv(
    "data/u1_base.txt", names=["user_id", "movie_id", "rating"], sep="\t"
)

ratings = ratings.merge(users_df, on="user_id")
print(f"Data: {len(users_df)} users, {len(films_df)} movies, {len(ratings)} ratings")


Data: 943 users, 1682 movies, 80000 ratings


In [187]:
for occ in users_df.occupation.unique():
    print(f'<option value="{occ}">{occ}</option>')

<option value="technician">technician</option>
<option value="other">other</option>
<option value="writer">writer</option>
<option value="executive">executive</option>
<option value="administrator">administrator</option>
<option value="student">student</option>
<option value="lawyer">lawyer</option>
<option value="educator">educator</option>
<option value="scientist">scientist</option>
<option value="entertainment">entertainment</option>
<option value="programmer">programmer</option>
<option value="librarian">librarian</option>
<option value="homemaker">homemaker</option>
<option value="artist">artist</option>
<option value="engineer">engineer</option>
<option value="marketing">marketing</option>
<option value="none">none</option>
<option value="healthcare">healthcare</option>
<option value="retired">retired</option>
<option value="salesman">salesman</option>
<option value="doctor">doctor</option>


In [75]:
# -------------------------
# Demographic recos
def recommend_me_demographic(sex, occupation, lower_age, upper_age, n):
    aux_ratings = ratings[ratings["occupation"] == occupation]
    aux_ratings = aux_ratings[aux_ratings["gender"] == sex]
    aux_ratings = aux_ratings[aux_ratings["age"] <= upper_age]
    aux_ratings = aux_ratings[aux_ratings["age"] >= lower_age]
    
    aux_ratings = (
        aux_ratings[["movie_id", "rating"]]
        .groupby("movie_id")
        .agg(count=("rating", "count"), mean=("rating", "mean"))
        .reset_index()
    )
    C = aux_ratings["mean"].mean()
    M = aux_ratings["count"].quantile(0.9)

    def weighted_rating(x, m=M, C=C):
        v = x["count"]
        R = x["mean"]
        # Calculation based on the IMDB formula
        return (v / (v + m) * R) + (m / (m + v) * C)
    aux_ratings["score"] = aux_ratings.apply(weighted_rating, axis=1)
    aux_ratings = aux_ratings.merge(films_df,on="movie_id")
    # print(aux_ratings.sort_values("score", ascending=False))
    return aux_ratings.sort_values("score")["title"].values[:n]

In [76]:
ratings[ratings["age"] < 25]

,user_id,movie_id,rating,age,gender,occupation
0,1,1,5,24,M,technician
1,1,2,3,24,M,technician
2,1,3,4,24,M,technician
3,1,4,3,24,M,technician
4,1,5,3,24,M,technician
...,...,...,...,...,...,...
79995,943,1067,2,22,M,student
79996,943,1074,4,22,M,student
79997,943,1188,3,22,M,student
79998,943,1228,3,22,M,student


In [77]:
def get_top_n_demographic(users,n):
    top_n = defaultdict(list)
    for user in users:
        recos = recommend_me_demographic(user,n)
        top_n[user]=recos
    return top_n

In [78]:

print(recommend_me_demographic("M","writer",30,40,5))


['Grease (1978)' 'Blues Brothers, The (1980)' 'Back to the Future (1985)'
 'Natural Born Killers (1994)' 'Star Wars (1977)']


In [46]:
a[a["age"]>10]

,user_id,movie_id,rating,age,gender,occupation
175,3,181,4,23,M,writer
176,3,258,2,23,M,writer
177,3,260,4,23,M,writer
178,3,268,3,23,M,writer
179,3,271,3,23,M,writer
...,...,...,...,...,...,...
75703,896,1471,1,28,M,writer
75704,896,1522,2,28,M,writer
75705,896,1622,2,28,M,writer
75706,896,1672,2,28,M,writer


In [33]:
get_top_n_demographic(users_df.user_id.unique(),5)

NameError: name 'defaultdict' is not defined

###  3. Collaborative Filtering
This system matches persons with similar interests and provides recommendations based on this matching. Collaborative filters do not require item metadata like its content-based counterparts. 

##### Example - 
If person A likes 3 movies, say Interstellar, Inception and Predestination, and person B likes Inception, Predestination and The Prestige, then they have almost similar interests. We can say with some certainty that A should like The Prestige and B should like Interstellar. 

The collaborative filtering algorithm uses “User Behavior” for recommending items. This is one of the most commonly used algorithms in the industry as it is not dependent on any additional information. 



The collaborative filtering can be modeled in any of the following ways:-

#### User-User collaborative filtering
It finds similarity scores between users to pick the most similar users and  recommends products which these similar users have liked or bought previously.

For movies, this algorithm finds the similarity between each user based on the ratings they have previously given to different movies. The prediction of an item for a user u is calculated by computing the weighted sum of the user ratings given by other users to an item i.
Thus the prediction for user 'u' is given as:

P(u,i) = E [r(v,i) * s(u,v)] / E [S(u,v)]

where,
* Pu,i is the prediction of an item
* Rv,i is the rating given by a user v to a movie i
* Su,v is the similarity between users

Basic steps for this-
* For predictions we need the similarity between the user u and v. We can make use of Pearson correlation.
* First we find the items rated by both the users and based on the ratings, correlation between the users is calculated.
* The predictions can be calculated using the similarity values. This algorithm, first of all calculates the similarity between each user and then based on each similarity calculates the predictions. Users having higher correlation will tend to be similar.


Disadvantage-
This algorithm is quite time consuming as it involves calculating the similarity for each user and then calculating prediction for each similarity score. One way of handling this problem is to select only a few users (neighbors) instead of all to make predictions, i.e. instead of making predictions for all similarity values, we choose only few similarity values. There are various ways to select the neighbors:

* Select a threshold similarity and choose all the users above that value
* Randomly select the users
* Arrange the neighbors in descending order of their similarity value and choose top-N users
* Use clustering for choosing neighbors



This algorithm is useful when the number of users is less. Its not effective when there are a large number of users as it will take a lot of time to compute the similarity between all user pairs. This leads us to item-item collaborative filtering, which is effective when the number of users is more than the items being recommended.


#### Item-Item collaborative filtering
Similarity is found between each items. Thus for movies similarities between movies is found and based on that recommendations of similar movies are made for the user. 

This algorithm works similar to user-user collaborative filtering with just a little change – instead of taking the weighted sum of ratings of “user-neighbors”, we take the weighted sum of ratings of “item-neighbors”.

##### What will happen if a new user or a new item is added in the dataset? 
It is called a Cold Start. 

* Visitor Cold Start
Visitor Cold Start means that a new user is introduced in the dataset. Since there is no history of that user, the system does not know the preferences of that user. It becomes harder to recommend products to that user. So, how can we solve this problem? One basic approach could be to apply a popularity based strategy, i.e. recommend the most popular products. These can be determined by what has been popular recently overall or regionally. Once we know the preferences of the user, recommending products will be easier.

*  Product Cold Start
Product Cold Start means that a new product is launched in the market or added to the system. User action is most important to determine the value of any product. More the interaction a product receives, the easier it is for our model to recommend that product to the right user. We can make use of Content based filtering to solve this problem. The system first uses the content of the new product for recommendations and then eventually the user actions on that product.

In [209]:
from collections import defaultdict


In [238]:
import pandas as pd 
import numpy as np 

In [241]:
movies = pd.read_csv("data/processed/movies.csv")


In [242]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),"[{'id': 3, 'name': 'Animation'}, {'id': 4, 'na..."
1,2,GoldenEye (1995),"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'..."
2,3,Four Rooms (1995),"[{'id': 16, 'name': 'Thriller'}]"
3,4,Get Shorty (1995),"[{'id': 1, 'name': 'Action'}, {'id': 5, 'name'..."
4,5,Copycat (1995),"[{'id': 6, 'name': 'Crime'}, {'id': 8, 'name':..."


In [215]:
from surprise import Reader, Dataset, SVD, KNNBasic
from surprise.model_selection import cross_validate

reader = Reader()

In [243]:


ratings = pd.read_csv("data/u1_base.txt",sep="\t",names=["user_id", "movie_id", "rating"])
ratings.head()

,user_id,movie_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [244]:
data = Dataset.load_from_df(ratings[["user_id", "movie_id", "rating"]], reader)

In [245]:
svd = KNNBasic()
cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=5)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.98845174, 0.98485375, 0.99251773, 0.98403372, 0.98865342]),
 'test_mae': array([0.77965187, 0.77911871, 0.78511417, 0.7794751 , 0.78250395]),
 'fit_time': (0.20955896377563477,
  0.2265329360961914,
  0.21914982795715332,
  0.21244001388549805,
  0.22269701957702637),
 'test_time': (1.9994728565216064,
  2.03222393989563,
  2.2869410514831543,
  2.0740349292755127,
  2.0117878913879395)}

In [246]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [247]:
data

In [248]:
recos = svd.test(trainset.build_testset())

In [233]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [(movie_id_title[x[0]],x[1]) for x in user_ratings[:n]] 

    return top_n

In [236]:
top_n["2"]

[]

In [234]:
top_n = get_top_n(recos, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

1 ['Star Wars (1977)', 'Empire Strikes Back, The (1980)', 'Maya Lin: A Strong Clear Vision (1994)', 'Amadeus (1984)', 'Wrong Trousers, The (1993)', 'Godfather, The (1972)', 'Godfather: Part II, The (1974)', 'Dead Man Walking (1995)', '12 Angry Men (1957)', 'Chasing Amy (1997)']
2 ['Godfather, The (1972)', 'Fargo (1996)', 'Secrets & Lies (1996)', 'L.A. Confidential (1997)', 'Good Will Hunting (1997)', 'Sense and Sensibility (1995)', 'Kolya (1996)', 'Full Monty, The (1997)', 'Emma (1996)', 'Leaving Las Vegas (1995)']
3 ['Paradise Lost: The Child Murders at Robin Hood Hills (1996)', 'Jackie Brown (1997)', 'Boogie Nights (1997)', 'Return of the Jedi (1983)', 'Wag the Dog (1997)', 'Mother (1996)', 'Apostle, The (1997)', "Ulee's Gold (1997)", 'L.A. Confidential (1997)', 'In the Name of the Father (1993)']
4 ['Contact (1997)', 'Air Force One (1997)', 'In & Out (1997)', 'Lost Highway (1997)', 'Assignment, The (1997)', 'Cop Land (1997)', 'Seven (Se7en) (1995)', 'Wonderland (1997)', 'Starship Tr

 For movies id = 654 we got estimated prediction of 3.83.
 
This recommender system doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

#### Conclusion
We create recommenders using demographic , content- based and collaborative filtering. While demographic filtering is very elemantary and cannot be used practically, Hybrid Systems can take advantage of content-based and collaborative filtering as the two approaches are proved to be almost complimentary. This model was very baseline and only provides a fundamental framework to start with.

References:
* https://www.kaggle.com/rounakbanik/movie-recommender-systems
* https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/#
        

In [169]:
import requests
import re

api = "4f829e75"

In [170]:
from tqdm import tqdm

In [207]:
def query_poster(recos):
    poster = []
    fails = []
    for reco in tqdm(recos):
        # Fetch Movie Data with Full Plot
        print(data_URL)
        title = re.sub(r'\(.*?\)', '', reco).strip()
        if(", The" in title):
            title= "The "+ title[:-5]            
        elif(", A" in title):
            title= "A  "+ title[:-3]        
        elif(", Il" in title):
            title= "Il "+ title[:-4]
            
        year = reco[-5:-1]
        params = {"t": title.strip(), "type": "movie"}
        response = requests.get(f"https://www.omdbapi.com/?apikey={api}", params=params).json()
        try:
            poster.append(response["Poster"])
        except:
            print(title, year)
            fails.append(title)
    print(response)
    return poster, fails

In [182]:
recos[0][-5:-1]

'1978'

In [183]:
recos[1][:-6].strip()

'Blues Brothers, The'

In [184]:
recos = recommend_me_demographic("M","writer",30,40,5)
query_poster(recos)

100%|██████████| 5/5 [00:00<00:00,  7.56it/s]


(['https://m.media-amazon.com/images/M/MV5BZmUyMDEyOTgtZmUwOS00NTdkLThlNzctNTM1ODQ4M2VhMjdhXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg',
  'https://m.media-amazon.com/images/M/MV5BYTdlMDExOGUtN2I3MS00MjY5LWE1NTAtYzc3MzIxN2M3OWY1XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg',
  'https://m.media-amazon.com/images/M/MV5BZmU0M2Y1OGUtZjIxNi00ZjBkLTg1MjgtOWIyNThiZWIwYjRiXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX300.jpg',
  'https://m.media-amazon.com/images/M/MV5BMTI2NTU2Nzc0MV5BMl5BanBnXkFtZTcwMzY1OTM2MQ@@._V1_SX300.jpg',
  'https://m.media-amazon.com/images/M/MV5BNzVlY2MwMjktM2E4OS00Y2Y3LWE3ZjctYzhkZGM3YzA1ZWM2XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg'],
 [])

In [185]:
query_poster(films_df.title.unique())

  2%|▏         | 40/1664 [00:07<10:37,  2.55it/s]

To Wong Foo, Thanks for Everything! Julie Newmar 1995


  3%|▎         | 45/1664 [00:10<12:05,  2.23it/s]

Eat Drink Man Woman 1994


  5%|▌         | 84/1664 [00:27<13:14,  1.99it/s]

...

Amistad 1997


 45%|████▍     | 745/1664 [06:16<06:10,  2.48it/s]

Tomorrow Never Dies 1997


 45%|████▍     | 745/1664 [06:16<07:44,  1.98it/s]


KeyboardInterrupt: 

In [195]:
api="591cd379"

In [208]:
t = "McHale's Navy (1997)"
query_poster([t])

  0%|          | 0/1 [00:00<?, ?it/s]

http://www.omdbapi.com/?apikey=591cd379


100%|██████████| 1/1 [00:00<00:00,  3.61it/s]

{'Title': "McHale's Navy", 'Year': '1997', 'Rated': 'PG', 'Released': '18 Apr 1997', 'Runtime': '108 min', 'Genre': 'Action, Comedy', 'Director': 'Bryan Spicer', 'Writer': 'Peter Crabbe, Andy Rose', 'Actors': 'Tom Arnold, Dean Stockwell, Ernest Borgnine', 'Plot': 'A retired Navy officer returns to active duty, along with a group of misfit sailors, to battle his former nemesis.', 'Language': 'English', 'Country': 'United States', 'Awards': '2 wins & 2 nominations', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMmU2NjcwYmMtZmZhYy00NjQ0LTkyZTktM2EwMWU2M2Y3ZmI4XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '4.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '3%'}, {'Source': 'Metacritic', 'Value': '18/100'}], 'Metascore': '18', 'imdbRating': '4.6', 'imdbVotes': '6,448', 'imdbID': 'tt0119640', 'Type': 'movie', 'DVD': '18 Mar 2014', 'BoxOffice': '$4,529,843', 'Production': 'The Bubble Factory', 'Website': 'N/A', 'Response': 'True'}


(['https://m.media-amazon.com/images/M/MV5BMmU2NjcwYmMtZmZhYy00NjQ0LTkyZTktM2EwMWU2M2Y3ZmI4XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX300.jpg'],
 [])